In [ ]:
import face_recognition
import cv2
import numpy as np
import os
import glob
import time
import tkinter as tk
from tkinter import ttk

user="Atul" # Full authorization to the user

video_capture = cv2.VideoCapture(0)

#make array of sample pictures with encodings
known_face_encodings = []
known_face_names = []
dirname = os.path.dirname(os.getcwd())
path = os.path.join(dirname, 'known_people/')

#make an array of all the saved jpg files' paths
list_of_files = [f for f in glob.glob(path+'*.jpg')]
#find number of known faces
number_files = len(list_of_files)

names = list_of_files.copy()

for i in range(number_files):
    globals()['image_{}'.format(i)] = face_recognition.load_image_file(list_of_files[i])
    globals()['image_encoding_{}'.format(i)] = face_recognition.face_encodings(globals()['image_{}'.format(i)])[0]
    known_face_encodings.append(globals()['image_encoding_{}'.format(i)])

    # Create array of known names
    names[i] = names[i].replace("known_people/", "")  
    known_face_names.append(names[i])
    known_faces_list = [name.split('\\')[-1].split('.')[0] for name in names]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

st_time = time.time()
while (time.time()<st_time+3):
    # Grab a single frame of video
    ret, frame = video_capture.read()
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name.split('\\')[-1].split('.')[0])

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)
    cv2.imwrite("Last_Login.jpg", frame)
    # Simply break if user himself is present in the frame
    if(user in face_names):
        break

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

# Printing the message as popup
def popupmsg(msg):
    popup = tk.Tk()
    popup.wm_title("Login Authorization")
    label = ttk.Label(popup, text=msg)
    label.pack(side="top", fill="x", pady=10, padx=100)
    B1 = ttk.Button(popup, text="OK", command = popup.destroy)
    B1.pack()
    popup.mainloop()

#pop-up and email to the user
def send_mail(message): 
    
    import smtplib

    s = smtplib.SMTP(host='smtp.gmail.com', port=587)
    s.starttls()

    MY_ADDRESS = "loginauth12@gmail.com"
    PASSWORD = "DdnJJJkMUR7hx5F"
    s.login(MY_ADDRESS, PASSWORD)
    to_email ="atulkumarkzy@gmail.com"

    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.image import MIMEImage
    msg = MIMEMultipart()       # create a message

        # setup the parameters of the message
    msg['From']=MY_ADDRESS
    msg['To']= to_email
    msg['Subject']="URGENT: Login authorization from MY-PC"
        # add in the message body
    msg.attach(MIMEText(message, 'plain'))
    fp = open("Last_Login.jpg", 'rb')
    msg.attach(MIMEImage(fp.read()))
        # send the message via the server set up earlier.
    s.send_message(msg)
    
    del msg

if(user in face_names):
    pop_msg = f"Welcome {user}"
    # Don't send email when the user himself is present in the frame
elif(any(known in face_names for known in known_faces_list)):
    present= list(filter(lambda known: known in known_faces_list, face_names))
    names_str = ", ".join(present)
    pop_msg = f"Welcome {names_str}. \n {user} will get the notification."
    message = f"{names_str} just opened your PC. The picture is attached with the mail."
    send_mail(message)
else:
    pop_msg = f"Unknown login: {user} will get a notification about this."
    message = "Someone unknown has opened your PC. The picture is attached with the mail."
    send_mail(message)
popupmsg(pop_msg)